## **Incident Clustering**

In [1]:
# UNCOMMENT AND RUN THIS THE FIRST TIME YOUR RUN THE SCRIPT, THEN COMMENT IT BACK
# Install packages to the correct location (3.12 site-packages)
import sys
import subprocess

print(f"Installing to: {sys.executable}")
print("Packages will go to: /home/jovyan/.conda/envs/default/lib/python3.12/site-packages")

subprocess.check_call([
    sys.executable, '-m', 'pip', 'install', '--force-reinstall',
    'numpy==2.2.4',
    'pandas==2.2.3', 
    'scikit-learn==1.6.1',
    'sentence-transformers==4.1.0',
    'torch==2.6.0',
    'transformers==4.51.3',
    'pycelonis==2.13.0'
])

Installing to: /home/jovyan/.conda/envs/default/bin/python
Packages will go to: /home/jovyan/.conda/envs/default/lib/python3.12/site-packages
Looking in indexes: https://pypi.celonis.cloud, https://pypi.org/simple
  Using cached numpy-2.2.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached https://pypi.celonis.cloud/pycelonis/pycelonis-2.13.0-py3-none-any.whl (255 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  U

0

In [2]:
# Importing required packages
from pycelonis import get_celonis
import pycelonis.pql as pql
from pycelonis.config import Config

import numpy as np
import pandas as pd
import re


from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import DBSCAN
import gc
from sentence_transformers import SentenceTransformer, util
import transformers
import torch

In [3]:
def assign_common_embedding(group):
    if group.shape[0] < 2: 
        return group
    common_embedding = group.iloc[0, df_number_of_cols:].values  # Use the first embedding in the group (all columns except 'ManufacturerPartNumber')
    for i in range(embeddings_reduced.shape[1]):
        group.iloc[:, df_number_of_cols+i] = common_embedding[i-1]
    return group

def hack_embeddings(column_name):
    df_withcol = df[df[column_name].notna()]
    df_withoutcol = df[df[column_name].isna()]
    df_withcol = df_withcol.groupby(column_name).apply(assign_common_embedding).reset_index(drop=True)
    return pd.concat((df_withcol, df_withoutcol), ignore_index=True)

#### **1.Get Data from Celonis**

In [4]:
# Get Data from Celonis

data_pool = 'OCPM Data Pool [ARM Multi-Object DEV]' #data pool where the data resides
data_model = 'perspective_custom_ITSM'

celonis = get_celonis()
pool = celonis.data_integration.get_data_pools().find(data_pool)
dm = pool.get_data_models().find(data_model)
Config.DEFAULT_DATA_MODEL = dm

# Choose if Data Model should be reloaded (True) or not (False)
UPDATE_DM = False

[2025-07-27 23:55:37,010] INFO: No `base_url` given. Using environment variable 'CELONIS_URL'
[2025-07-27 23:55:37,011] INFO: No `api_token` given. Using environment variable 'CELONIS_API_TOKEN'


[2025-07-27 23:55:37,195] WARNING: Your PyCelonis Version 2.13.0 is outdated (Newest Version: 2.14.0). Please upgrade the package via: pip install --extra-index-url=https://pypi.celonis.cloud/ pycelonis pycelonis_core --upgrade
[2025-07-27 23:55:37,398] WARNING: KeyType is not set. Defaulted to 'APP_KEY'.


[2025-07-27 23:55:37,437] INFO: Initial connect successful! PyCelonis Version: 2.13.0
[2025-07-27 23:55:37,531] INFO: `ml-workbench` permissions: ['DELETE_SCHEDULERS', 'EDIT_SCHEDULERS', 'USE_ALL_SCHEDULERS', 'CREATE_APPS', 'USE_ALL_APPS', 'CREATE_SCHEDULERS', 'CREATE_WORKSPACES', 'MANAGE_ALL_APPS', 'MANAGE_ALL_WORKSPACES', 'MANAGE_SCHEDULERS_PERMISSIONS', 'VIEW_CONFIGURATION']
[2025-07-27 23:55:37,531] INFO: `team` permissions: []
[2025-07-27 23:55:37,532] INFO: `process-repository` permissions: []
[2025-07-27 23:55:37,532] INFO: `storage-manager` permissions: []
[2025-07-27 23:55:37,532] INFO: `package-manager` permissions: []
[2025-07-27 23:55:37,532] INFO: `transformation-center` permissions: []
[2025-07-27 23:55:37,533] INFO: `compute-live` permissions: []
[2025-07-27 23:55:37,533] INFO: `task-mining` permissions: []
[2025-07-27 23:55:37,533] INFO: `event-collection` permissions: ['EDIT_ALL_DATA_POOLS_RESTRICTED', 'USE_ALL_DATA_MODELS', 'VIEW_ALL_DATA_POOLS', 'CREATE_DATA_POOL', '

In [5]:
# Shows in the DM except event tables, relationship tables and tables starting with _
print(f"Table Analysis:")
for tab in dm.get_tables():
    if not (tab.name.startswith('e') or tab.name.startswith('r') or tab.name.startswith('_')):
        print(tab.alias)

print(f"\n")

# Input the tables aliases as they are in the DM
table_aliases = ['o_custom_RequestItem']
tables = {table_alias:[x for x in dm.get_tables() if x.alias == table_alias][0] for table_alias in table_aliases} 
tables

# Inspect columns
columns = {table_name:table.get_columns() for table_name,table in tables.items()}

print(f"Column Analysis:")
for table_name, cols in columns.items():
    print(table_name + f" columns ({len(cols)} total):\n")
    for col in cols:
        print(col.name)
    print(f"\n")

Table Analysis:
o_custom_RequestItem
o_custom_Sla
o_custom_AssignmentGroup
o_custom_Incident
o_custom_CatalogTask
o_custom_Problem
o_custom_ProblemTask
o_custom_Problem__AssignmentGroup
o_custom_CatalogTask__Sla
o_custom_Cluster


Column Analysis:
o_custom_RequestItem columns (28 total):

Cluster
PrimaryCatalog
Number
RequestNumber
AssignmentGroupName
AssignedTo
ReassignmentCount
OpenedBy
RequestedFor
CreatedBy
ClosedBy
CatalogItem
ConfigurationItem
BusinessService
ContactType
Location
ShortDescription
Description
Priority
Impact
Urgency
DueDate
Approval
ApprovalSet
State
OpenedAt
ClosedAt
ID




In [6]:
pql_filter = '''FILTER UPPER("o_custom_RequestItem"."CatalogItem") LIKE ('%NON-CATALOG IT%') AND DAYS_BETWEEN("o_custom_RequestItem"."OpenedAt", MINUTE_NOW()) <= 365'''    # change this if you want to include a filter

# there are two ways to get data: either via simple columns/attributes or via custom PQL
# simple column example: columns['MARA'].find("MANDT") would give you MARA.MANDT
df_meta = pql.DataFrame(
    data=
    {
        "ID": columns['o_custom_RequestItem'].find("ID")
        ,"freetext": columns['o_custom_RequestItem'].find("ShortDescription")
    },
    filters=
    {pql_filter}
)

try:
    df = df_meta.to_pandas().reset_index(drop=True)
except:
    df = pd.read_csv('materials-dataset.csv')
    df.columns = ['ID','freetext']
    
df = df.drop_duplicates().dropna(subset=['ID'])

print(f"\n")
print(f"Memory usage: {df.memory_usage().sum() / 1e6: .2f} MB")
print("Nr. of records:", len(df))
print("Nr. of unique materials:", len(df["ID"].unique()))
print(f"\n")
df.head(10)

[2025-07-27 23:55:39,189] INFO: Successfully created data export using api v1 with id '1c63ddf0-8b9a-4626-bbc3-9a0df3b65571'
[2025-07-27 23:55:39,189] INFO: Wait for execution of data export with id '1c63ddf0-8b9a-4626-bbc3-9a0df3b65571'


0it [00:00, ?it/s]

[2025-07-27 23:55:39,269] INFO: Export result chunks for data export with id '1c63ddf0-8b9a-4626-bbc3-9a0df3b65571'


Memory usage:  0.20 MB
Nr. of records: 12489
Nr. of unique materials: 12489




,ID,freetext
0,170cc3c761b5ea1040babaa27115c89c,Please give access to remote windows machine
1,171eed501b82d2102380404cd34bcbc8,Need access to AI office ServiceNow group - Query
2,17c7ffe51b48de907abf9828b04bcb3f,Request for Apple Business Manager access
3,17ccd1029f0226107ed453e76724ab68,Assistance in the Multiverse navigation
4,18bce85c1b1f5a108fb2337f034bcbf0,How to reverse a free-issuing of stock
5,18c78226c384a6501c44d8ed05013150,Assistance for MS Project installation
6,18ccdadac3b466d8661799fb0501319f,Service Account nnx_mbed is not reflecting in ...
7,18da26e483182ed08a4fed60ceaad3d8,Set up OOO Automatic reply Agisti Reddy
8,196ba89083c7a2d03d06e8b52bda1ecd,Multiverse - Would Like to Request a Rule to ...
9,196c70621bdb161468398557d34bcb24,Tech Hub on call rota


#### **2. Data Preparation**

In [7]:
# Prepare freetext column to be fed to the LLM (i.e. set the freetext column as a concatentation of columns from the data above, use the column names as in row[column_name])
df["freetext"] = df.apply(lambda row: f"{row['freetext']}", axis=1)
df['freetext'] = df['freetext'].fillna('')
#df

# Needed for correct concatenation of columns later
df_number_of_cols = df.shape[1]
#df_number_of_cols

#### **3. Clustering using Embeddings + Additional Features**

In [8]:
# Run transformer
model = SentenceTransformer("sentence-transformers/LaBSE")  # previously: all-MiniLM-L6-v2
embeddings = model.encode(df.freetext, batch_size=256, convert_to_tensor=True, show_progress_bar=True)
print("Embeddings shape: ", embeddings.shape)

Batches:   0%|          | 0/49 [00:00<?, ?it/s]

Embeddings shape:  torch.Size([12489, 768])


In [16]:
# Test parameters (memory-safe approach)
import pandas as pd
import gc
import numpy as np

print("Testing parameters...")
results = []

# Test fewer combinations to avoid kernel crash
for min_samp in [2, 3]:  # Just test 2 and 3
    for n_components in [75, 100, 125]:  # Added 75, removed 150 to save memory
        pca = PCA(n_components=n_components)
        embeddings_reduced = pca.fit_transform(embeddings)
        
        for eps_val in [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.65, 0.7]:  # Added 0.35 and 0.45
            algo = DBSCAN(min_samples=min_samp, eps=eps_val)
            clusters = algo.fit_predict(embeddings_reduced)
            
            noise_pct = (clusters == -1).sum() / len(clusters) * 100
            n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            variance_explained = pca.explained_variance_ratio_.sum()
            
            # Calculate average cluster size
            if n_clusters > 0:
                avg_cluster_size = (len(clusters) - (clusters == -1).sum()) / n_clusters
            else:
                avg_cluster_size = 0
            
            # Calculate cluster size distribution metrics
            if n_clusters > 0:
                cluster_values = pd.Series(clusters)
                cluster_sizes = cluster_values.value_counts()
                cluster_sizes = cluster_sizes[cluster_sizes.index != -1]  # Exclude noise
                
                if len(cluster_sizes) > 0:
                    tiny_clusters = (cluster_sizes <= 5).sum()
                    tiny_pct = tiny_clusters / len(cluster_sizes) * 100
                    median_cluster_size = cluster_sizes.median()
                    largest_cluster = cluster_sizes.max()
                    # Calculate cluster size standard deviation (measure of distribution balance)
                    cluster_size_std = cluster_sizes.std()
                else:
                    tiny_pct = 0
                    median_cluster_size = 0
                    largest_cluster = 0
                    cluster_size_std = 0
            else:
                tiny_pct = 0
                median_cluster_size = 0
                largest_cluster = 0
                cluster_size_std = 0
                
            results.append({
                'min_samples': min_samp,
                'pca_components': n_components,
                'eps': eps_val,
                'noise_pct': round(noise_pct, 1),
                'n_clusters': n_clusters,
                'avg_cluster_size': round(avg_cluster_size, 1),
                'tiny_cluster_pct': round(tiny_pct, 1),
                'median_cluster_size': median_cluster_size,
                'largest_cluster': largest_cluster,
                'cluster_size_std': round(cluster_size_std, 1),
                'variance_explained': round(variance_explained, 3)
            })
            
            # Clean up memory after each test
            del clusters, algo
            gc.collect()
        
        # Clean up after each PCA test  
        del embeddings_reduced, pca
        gc.collect()

# Create DataFrame and find optimal settings
results_df = pd.DataFrame(results)

print("\n" + "="*80)
print("CLUSTERING RESULTS:")
print("="*80)
print(results_df.to_string(index=False))

# GENERALIZED quality score - works across different dataset sizes and types
total_items = len(embeddings)

# Calculate relative thresholds based on dataset size
ideal_clusters = max(10, min(1000, int(total_items ** 0.5)))  # Square root scaling
ideal_avg_size = max(5, min(200, total_items / ideal_clusters))  # Reasonable cluster sizes

results_df['quality_score'] = (
    # Penalize extreme noise (too high OR too low) - use np.minimum instead of min
    np.minimum(abs(results_df['noise_pct'] - 8), abs(results_df['noise_pct'] - 15)) * 2 +
    
    # Penalize very high tiny cluster percentage (over 50% is usually bad)
    np.maximum(0, results_df['tiny_cluster_pct'] - 50) * 3 +
    
    # Penalize extreme cluster counts (too few or too many relative to data size)
    (abs(results_df['n_clusters'] - ideal_clusters) / ideal_clusters) * 50 +
    
    # Prefer higher variance explained
    (1 - results_df['variance_explained']) * 100 +
    
    # Penalize very unbalanced distributions (high std relative to mean)
    np.where(results_df['avg_cluster_size'] > 0, 
             (results_df['cluster_size_std'] / results_df['avg_cluster_size']) * 20, 0) +
    
    # Penalize median cluster size being too small (indicates over-clustering)
    np.maximum(0, 8 - results_df['median_cluster_size']) * 3 +
    
    # HEAVILY penalize monster clusters (over 1000 incidents is usually bad)
    np.maximum(0, (results_df['largest_cluster'] - 1000) / 1000) * 100 +
    
    # Additional penalty for any cluster over 5% of total data
    np.maximum(0, (results_df['largest_cluster'] / total_items * 100 - 5)) * 10
)

# Find and set optimal parameters
best_idx = results_df['quality_score'].idxmin()
best_settings = results_df.loc[best_idx]

# AUTO-SET OPTIMAL PARAMETERS
optimal_min_samples = int(best_settings['min_samples'])
optimal_pca_components = int(best_settings['pca_components'])
optimal_eps = best_settings['eps']

print("\n" + "="*80)
print("🏆 OPTIMAL PARAMETERS SET:")
print("="*80)
print(f"Dataset size: {total_items:,} items")
print(f"Ideal clusters (√n): {ideal_clusters}")
print(f"min_samples: {optimal_min_samples}")
print(f"PCA Components: {optimal_pca_components}")
print(f"DBSCAN eps: {optimal_eps}")
print(f"Expected noise: {best_settings['noise_pct']}%")
print(f"Expected clusters: {int(best_settings['n_clusters'])}")
print(f"Expected tiny cluster %: {best_settings['tiny_cluster_pct']}%")
print(f"Expected median cluster size: {best_settings['median_cluster_size']}")
print(f"Expected largest cluster: {best_settings['largest_cluster']}")
print("="*80)

Testing parameters...

CLUSTERING RESULTS:
 min_samples  pca_components  eps  noise_pct  n_clusters  avg_cluster_size  tiny_cluster_pct  median_cluster_size  largest_cluster  cluster_size_std  variance_explained
           2              75 0.20       85.4         588               3.1              92.2                  2.0               30               3.1               0.753
           2              75 0.25       84.1         621               3.2              90.2                  2.0               31               3.2               0.753
           2              75 0.30       82.2         679               3.3              90.0                  2.0               46               3.6               0.753
           2              75 0.35       79.8         757               3.3              90.0                  2.0               46               3.8               0.753
           2              75 0.40       76.9         843               3.4              89.4                  2.

In [20]:
# Run PCA with optimal parameters
pca = PCA(n_components=optimal_pca_components)
embeddings_reduced = pca.fit_transform(embeddings)
print(f"Optimal EPS Set: {optimal_eps}")
print("PCA explained variance ratio: ", pca.explained_variance_ratio_.sum())
print("Embeddings shape:", embeddings_reduced.shape)
gc.collect()

Optimal EPS Set: 0.3
PCA explained variance ratio:  0.8997188299674871
Embeddings shape: (12489, 125)


5748

In [21]:
# Add features for digits to reduce false positive rate, i.e. each digit is a feature
# addfeatures = df['freetext'].apply(lambda x: ','.join(re.findall(r'\d', x))).str.split(',', expand=True).fillna(0).replace('',0).astype(int).values

# NUMBERS_WEIGHTING = 5
# embeddings_reduced = np.concatenate((embeddings_reduced[:,:-10], NUMBERS_WEIGHTING*addfeatures), axis=1)

In [ ]:
df = pd.concat([df, pd.DataFrame(embeddings_reduced, columns=[f'emb_{i}' for i in range(embeddings_reduced.shape[1])])], axis=1)

Hack Embeddings (Optional)

In [ ]:
# this section does the following: in case there is a column in the dataset that dictates two records to be considered the same, then their embeddings will be set equal
# that way, they end up being in the same cluster
# Example: records that have been validated by the customer to be in the same cluster will continue being in the same cluster in further iterations
# so if there is a column called ValidationFlag that captures that, we would call hack_embeddings('ValidationFlag')

# df = hack_embeddings('ManufacturerPartNumber')
# df = hack_embeddings('OldMaterialNumber')
# df = hack_embeddings('ValidationFlag')

In [ ]:
### Run clustering algorithm: DBSCAN with optimal parameters
print(f"Optimal EPS Set: {optimal_eps}")
print(f"Optimal Min Samples Set: {optimal_min_samples}")
algo = DBSCAN(min_samples=optimal_min_samples, eps=optimal_eps)
clusters = algo.fit_predict(embeddings_reduced)
# Add clusters to dataframe
df['cluster'] = clusters.astype(str)
df['pca1'] = embeddings_reduced[:,0]
df['pca2'] = embeddings_reduced[:,1]
# Calculate clustering statistics
total_incidents = len(clusters)
noise_incidents = (clusters == -1).sum()
clustered_incidents = total_incidents - noise_incidents
unique_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)  # Exclude noise cluster from count
print(f"Nr. unique clusters: {unique_clusters}")
print(f"Total incidents: {total_incidents:,}")
print(f"Incidents successfully clustered: {clustered_incidents:,} ({clustered_incidents/total_incidents*100:.1f}%)")
print(f"Noise incidents (outliers): {noise_incidents:,} ({noise_incidents/total_incidents*100:.1f}%)")
df
# Show potential duplicates
cluster_multiple_materials = df['cluster'].value_counts().index[(df['cluster'].value_counts() > 1)]
duplicates = df[df['cluster'].isin(cluster_multiple_materials)].sort_values('cluster')

In [ ]:
# Text-based cluster size analysis (no imports needed)
import pandas as pd

# Get cluster sizes (excluding noise)
cluster_sizes = df['cluster'].value_counts()
cluster_sizes = cluster_sizes[cluster_sizes.index != '-1']  # Exclude noise

print(f"Cluster size statistics:")
print(f"Total clusters: {len(cluster_sizes)}")
print(f"Smallest cluster: {cluster_sizes.min()}")
print(f"Largest cluster: {cluster_sizes.max()}")
print(f"Median cluster size: {cluster_sizes.median()}")
print(f"Mean cluster size: {cluster_sizes.mean():.1f}")

# Create size buckets
size_buckets = {
    'Tiny (2-5)': (cluster_sizes <= 5).sum(),
    'Small (6-20)': ((cluster_sizes > 5) & (cluster_sizes <= 20)).sum(),
    'Medium (21-100)': ((cluster_sizes > 20) & (cluster_sizes <= 100)).sum(),
    'Large (101-500)': ((cluster_sizes > 100) & (cluster_sizes <= 500)).sum(),
    'Huge (500+)': (cluster_sizes > 500).sum()
}

print(f"\n{'='*60}")
print(f"CLUSTER SIZE DISTRIBUTION:")
print(f"{'='*60}")

for bucket, count in size_buckets.items():
    pct = count / len(cluster_sizes) * 100
    bar = '█' * int(pct / 2)  # Simple text bar chart
    print(f"{bucket:15} | {count:4d} clusters ({pct:5.1f}%) {bar}")

print(f"\n{'='*60}")
print(f"TOP 10 LARGEST CLUSTERS:")
print(f"{'='*60}")

top_clusters = cluster_sizes.head(10)
for i, (cluster_id, size) in enumerate(top_clusters.items(), 1):
    print(f"{i:2d}. Cluster {cluster_id}: {size:,} incidents")

# Show some example incidents from the largest cluster
print(f"\n{'='*60}")
print(f"SAMPLE INCIDENTS FROM LARGEST CLUSTER ({top_clusters.iloc[0]} incidents):")
print(f"{'='*60}")

largest_cluster_id = top_clusters.index[0]
largest_cluster_incidents = df[df['cluster'] == largest_cluster_id]['freetext'].head(5)

for i, incident in enumerate(largest_cluster_incidents, 1):
    print(f"{i}. {incident[:100]}..." if len(incident) > 100 else f"{i}. {incident}")

#### **4. Push Data Back and Update DM**

In [21]:
table_name = 'CatalogItemClusters'

pool.create_table(
    df.drop(columns=[col for col in df.columns if 'emb' in col]), 
    table_name=table_name, 
    drop_if_exists=True, 
    force=True)

if UPDATE_DM:
    dm.reload(wait=False)

[2025-07-21 20:25:40,758] WARNING: STRING columns are by default stored as VARCHAR(80) and therefore cut after 80 characters. You can specify a custom field length for each column using the `column_config` parameter.


[2025-07-21 20:25:40,928] INFO: Successfully created data push job with id 'e4f64c2f-0b2b-410f-81a3-ad1153fab0e7'
[2025-07-21 20:25:40,929] INFO: Add data frame as file chunks to data push job with id 'e4f64c2f-0b2b-410f-81a3-ad1153fab0e7'


  0%|          | 0/1 [00:00<?, ?it/s]

[2025-07-21 20:25:41,340] INFO: Successfully upserted file chunk to data push job with id 'e4f64c2f-0b2b-410f-81a3-ad1153fab0e7'
[2025-07-21 20:25:41,693] INFO: Successfully triggered execution for data push job with id 'e4f64c2f-0b2b-410f-81a3-ad1153fab0e7'
[2025-07-21 20:25:41,694] INFO: Wait for execution of data push job with id 'e4f64c2f-0b2b-410f-81a3-ad1153fab0e7'


0it [00:00, ?it/s]

[2025-07-21 20:25:47,888] INFO: Successfully created table 'incidentClusters' in data pool
[2025-07-21 20:25:48,021] INFO: Successfully deleted data push job with id 'e4f64c2f-0b2b-410f-81a3-ad1153fab0e7'
